In [18]:
from backtest_lib.engine.decision import *
from backtest_lib.engine import make_engine
from backtest_lib.engine.execute.perfect_world import *
from backtest_lib.engine.plan.perfect_world import *
from backtest_lib.portfolio import cash, uniform_portfolio
import random

universe = (
    "security1",
    "security2",
    "security3",
    "security4",
    "security5",
)

prices = make_universe_mapping(
    {sec: random.random() * 1000 for sec in universe},
    universe=universe,
    constructor_backend="polars",
)

print(f"Set prices to {tuple(prices.items())}")

portfolio = uniform_portfolio(universe, value=100000, backend="polars")


def strategy_pro_rata(universe, market, current_portfolio, ctx):
    return target_weights({"security1": 0.5, "security2": 0.5}) + reallocate(
        0.2, out_of=("security1", "security2"), into=("security3", "security4")
    )


def strategy_equal(universe, market, current_portfolio, ctx):
    return target_weights({"security1": 0.5, "security2": 0.5}) + reallocate(
        0.2,
        out_of=("security1", "security2"),
        into=("security3", "security4"),
        mode="equal_out_equal_in",
    )


planner = PerfectWorldPlanGenerator()
executor = PerfectWorldPlanExecutor(backend="polars", security_alignment=universe)

engine = make_engine(planner, executor, universe)

Set prices to (('security1', 32.783931515227735), ('security2', 192.63064651399387), ('security3', 424.84089373951883), ('security4', 904.0628563008613), ('security5', 31.35238763757231))


In [19]:
plan = planner.generate_plan(decision, prices)
executor.execute_plan(plan=plan, portfolio=portfolio, prices=prices, market=None)

mode: pro_rata_out_equal_in
45.08291560584433
265.78075000807604
here
figured out allocation and target, combining
type(reallocation) : <class 'backtest_lib.engine.execute.perfect_world._WeightsReallocation'>
type(target) : <class 'backtest_lib.engine.execute.perfect_world._TargetWeightsCompiledOp'>
applying reallocation...
(('security1', -0.029087676406605603), ('security2', -0.1709123235933944), ('security3', 0.1), ('security4', 0.1), ('security5', 0.0))
sum: 0.0


ExecutionResult(before=<backtest_lib.portfolio.WeightedPortfolio object at 0x7fc3df2b2810>, after=<backtest_lib.portfolio.WeightedPortfolio object at 0x7fc3dcce0710>, costs=CostBreakdown(fees=0, slippage=0), fills=None, warnings=())

In [20]:
execution_result_pro_rata = engine.execute_strategy(
    strategy=strategy_pro_rata,
    portfolio=portfolio,
    market=None,
    ctx=None,
    prices=prices,
)
execution_result.after.holdings

mode: pro_rata_out_equal_in
45.08291560584433
265.78075000807604
here
figured out allocation and target, combining
type(reallocation) : <class 'backtest_lib.engine.execute.perfect_world._WeightsReallocation'>
type(target) : <class 'backtest_lib.engine.execute.perfect_world._TargetWeightsCompiledOp'>
applying reallocation...
(('security1', -0.029087676406605603), ('security2', -0.1709123235933944), ('security3', 0.1), ('security4', 0.1), ('security5', 0.0))
sum: 0.0


SeriesUniverseMapping(names=('security1', 'security2', 'security3', 'security4', 'security5'), _data=shape: (5,)
Series: '' [f64]
[
	0.365639
	0.434361
	0.1
	0.1
	0.0
], _scalar_type=<class 'float'>)

In [21]:
execution_result_equal = engine.execute_strategy(
    strategy=strategy_equal, portfolio=portfolio, market=None, ctx=None, prices=prices
)

mode: equal_out_equal_in
figured out allocation and target, combining
type(reallocation) : <class 'backtest_lib.engine.execute.perfect_world._WeightsReallocation'>
type(target) : <class 'backtest_lib.engine.execute.perfect_world._TargetWeightsCompiledOp'>
applying reallocation...
(('security1', -0.1), ('security2', -0.1), ('security3', 0.1), ('security4', 0.1), ('security5', 0.0))
sum: 0.0
